In [3]:
# Deep Learning Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

# Data Manipulation and Analysis
import numpy as np
import pandas as pd
import collections # A module providing alternative data structures like named tuples, defaultdict, Counter, etc., compared to built-in Python containers.
import random

# Data Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn

# File and System Interaction
import glob
import os
from pathlib import Path
import shutil

# Scientific Computing and Math
import math
import cmath

# Date and Time Handling
import time
import datetime

# Linear Algebra
from torch import linalg as LA

In [ ]:
# # Get scripts from github

# # Download it from GitHub
# try:
#   from py_scripts import convmc, dataset_processing, inference, logs_and_results, training
# except:
#   # Clone github to access data
#   !git clone https://github.com/TalhaAhmed2000/convmc-net.git
#   !mv convmc-net/python_scripts py_scripts
#   from py_scripts import convmc, dataset_processing, inference, logs_and_results, training

In [ ]:
# !rm -r convmc-net
# !rm -r py_scripts

## Dataset.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug  3 13:29:11 2018

@author: Yi Zhang
"""



class Dataset:
    def __init__(self,folder=None,shuffle=None,prefix=None,shownum=10):
        self._folder=None
        if not folder is None:
            self._folder=folder
            self._flist=os.listdir(folder)
        if not prefix is None:
            tmp=[]
            for s in self._flist:
                if s.startswith(prefix):
                    tmp.append(s)
            self._flist=tmp
        self._flist.sort()
        if not shuffle is None:
            random.shuffle(self._flist,shuffle)
            print(self._flist[0:shownum])

    def getnpz(self,fileind,arrind,folder=None):
        if folder is None:
            folder=self._folder
            flist=self._flist
        else:
            flist=os.listdir(folder)
        data=np.load(folder+flist[fileind],mmap_mode='r+')
        dlist=[]
        l=0
        for x in data:
            if l in arrind:
                dlist.append(data[x])
            l=l+1
        return dlist

    def getmat(self,fileind,arrind,folder=None):
        if folder is None:
            folder=self._folder
            flist=self._flist
        else:
            flist=os.listdir(folder)
        data=loadmat(folder+flist[fileind])
        dlist=[]
        for x in arrind:
            dlist.append(data[x])
        return dlist


class Converter:
    def __init__(self):
        """
        Preprocessing:
        1.concat: inv=False, concatenate real and imaginary parts in axis=-1
            inv=True, depart two parts in -1 axis into real and imaginary parts
        2.ch2: inv=False, concatenate real and imaginary parts in a new axis=0
            inv=True, depart two parts in 1 axis into real and imaginary parts
        3.stack: inv=False, stack real and imaginary parts in axis=-1
            inv=True, depart two parts in -1 axis into real and imaginary parts
        3.None: pass
        """
        # self.pre={'concat':self.concat,'ch2':self.ch2,'stack':self.stack}

    def np2torch(self,xnp,formlist):
        dlist=[]
        for i,x in enumerate(xnp):
            # if ('pre' in formlist[i]) and (formlist[i]['pre'] in self.pre):
            #     x=self.pre[formlist[i]['pre']](x,inv=False)
            x=x.reshape(formlist[i]['shape'])
            x=torch.tensor(x,dtype=torch.float32)
            dlist.append(x)
        return dlist

    def torch2np(self,xtorch,formlist):
        dlist=[]
        for i,x in enumerate(xtorch):
            if torch.cuda.is_available():
                x=x.cpu().detach().numpy()
            else:
                x=x.detach().numpy()
            # if ('pre' in formlist[i]) and (formlist[i]['pre'] in self.pre):
                # print([formlist[i]['pre']])
                # print(self.pre)
                # x=self.pre[formlist[i]['pre']](x,inv=False)
            # x=x.reshape(formlist[i]['shape'])
            dlist.append(x)
        return dlist

    # def stack(self,x,inv):
        # if not inv:
            # #not inv, np2torch
            # x=np.swapaxes(x,0,-1)
            # xr=x.real
            # xi=x.imag
            # size=list(x.shape)
            # size[0]*=2
            # size=tuple(size)
            # z=np.zeros(size)
            # z[0::2]=xr
            # z[1::2]=xi
            # z=np.swapaxes(z,0,-1)
        # else:
            #inv, torch2np
            # #size=[B,C,H,W,(T)],numpy
            # x=np.swapaxes(x,0,-1)
            # xr=x[0::2]
            # xi=x[1::2]
            # z=xr+1j*xi
            # z=np.swapaxes(z,0,-1)

        return x

    # def concat(self,x,inv):
        # if not inv:
            #not inv, np2torch
            # z=np.concatenate((x.real,x.imag),axis=-1)
        # else:
            #inv, torch2np
            #size=[B,C,H,W,(T)],numpy
            # x=np.swapaxes(x,0,-1)
            # n=x.shape[0]
            # nh=int(n/2)
            # xr=x[0:nh]
            # xi=x[nh:n]
            # z=xr+1j*xi
            # z=np.swapaxes(z,0,-1)

        return z

    def ch2(self,x,inv):
        pass


## Player.py

In [ ]:
# Player.py

# -*- coding: utf-8 -*-
"""
Created on Thu Aug  2 15:43:10 2018

@author: Yi Zhang
"""

# import numpy as np
# import matplotlib.pyplot as plt
# import torch

minDBdf=-50
class Player:
    def __init__(self,Tpause=None):
        if Tpause==None:
            self.Tpause=0.1
        else:
            self.Tpause=Tpause
        self.fig=None
        self.ax={1:None,2:None,3:None,4:None,5:None,6:None}
        self.axnum=0

    def plotmat(self,mvlist,note=None,tit=None,supt=None,
                cmap='gray',ion=True,minDB=None):
        """
        input:matrix dimension
        """
        if minDB is None:
            minDB=minDBdf

        if ion:
            plt.ion()

        # print('In function plotmat')
        subp={1:[1,1],2:[1,2],3:[1,3],4:[2,2],5:[2,3],6:[2,3],9:[3,3],12:[4,3]}
        # print('len(mvlist)',len(mvlist))
        p1,p2=subp[len(mvlist)]
        # print('p1,p2',p1,p2)
        if p1*p2!=self.axnum or self.fig is None or not(plt.fignum_exists(self.fig.number)):
            'in this if statement'
            self.fig,(self.ax)=plt.subplots(p1,p2)
            self.axnum=p1*p2
            if self.axnum==1:
                self.ax=np.array([self.ax])
            self.ax=self.ax.reshape([-1])

        for i in range(len(mvlist)):
            US=mvlist[i]
            # print('i of mvlist',i)
            if US is None:
                continue
            if US.dtype is torch.float32:
                US=US.detach().numpy()
                # 'in this if statement 2'
            # print ('US.shape',US.shape)
            US=np.abs(US) #.reshape([-1,mvlist[i].shape[-1]])
            # US = np.transpose(US, (1, 2, 0))
            if np.sum(np.abs(US))!=0:
                US=US/np.max(US)
            if note=='db':
                US[US<10**(minDB/20)]=10**(minDB/20)
                US=20*np.log10(US)
            vmin,vmax=[minDB,0] if note=='db' else [0,1]
            self.ax[i].clear()
            self.ax[i].imshow(US)
            if not(tit is None):
                self.ax[i].set_title(tit[i])
        if not(supt is None):
            self.fig.suptitle(supt)
        if ion:
            plt.pause(self.Tpause)
        return self.fig

    def play(self,mvlist,note=None,tit=None,supt=None,cmap='gray',minDB=None):
        """
        input:movie dimension
        """
        if minDB is None:
            minDB=minDBdf

        subp={1:[1,1],2:[1,2],3:[1,3],4:[2,2],5:[2,3],6:[2,3],9:[3,3]}
        p1,p2=subp[len(mvlist)]
        T=mvlist[0].shape[-1]
        fig,ax=plt.subplots(p1,p2)
        if p1*p2==1:
            ax=np.array([ax])
        ax=ax.reshape([-1])

        plt.ion()

        for i in range(len(mvlist)):
            US=mvlist[i]
            if US is None:
                continue
            if US.dtype is torch.float32:
                US=US.detach().numpy().squeeze()

            US=np.abs(US)
            if np.sum(np.abs(US))!=0:
                US=US/np.max(US)
            if note=='db':
                US[US<10**(minDB/20)]=10**(minDB/20)
                US=20*np.log10(US)
            mvlist[i]=US

        for t in range(T):
            for i in range(len(mvlist)):
                if mvlist[i] is None:
                    continue
                vmin,vmax=[minDB,0] if note=='db' else [0,1]
                ax[i].clear()
                ax[i].imshow(mvlist[i][:,:,t],cmap=cmap,aspect='auto',
                             vmin=vmin,vmax=vmax)
                if not(tit is None):
                    ax[i].set_title(tit[i])
            if supt==None:
                supt=''
            fig.suptitle('%dth Frame,'%(t+1)+supt)
            plt.pause(self.Tpause)


## My UnfoldedNet2dC.py

# Colab

#### DataSet_Unfolded for Real World Sensing Data.py

In [ ]:
# Setting up some global variables

ROOT = '/content/drive/MyDrive/DUPA - RCPA/Technology transfer deep unfolding/SPROJ-ConvMC-Net/Sensor Data'
TRY = '5st try'
SESSION = 'Session 5'

#### Model path of loading

Testing Training Loop

In [ ]:
# Get parameters
def get_default_param(hyper_param_net, gpu = True):
    params_net = {}
    if hyper_param_net['Model'] == 'ADMM-Net':
        params_net['layers'] = 5

        params_net['initial_neta'] = 1.81    # fixed
        params_net['initial_lamda1'] = 0.051 # fixed
        params_net['initial_lamda2'] = 0.049 # fixed
        params_net['initial_v'] = 0

        params_net['initial_S'] = 0.05001 #fixed

        params_net['initial_P'] = 0.2401 #fixed

        params_net['initial_rho'] = 0.1001

        params_net['coef_gamma'] = 0.4001

        params_net['CalInGPU'] = gpu #whether to calculate in GPU
        params_net['size1'] = 400
        params_net['size2'] = 500

    else:
        params_net['layers'] = 5 #1 #2 #3 #4 #5 #6 #7 #8 #9 #10
        params_net['kernel'] = [(3, 1)] * 3 + [(3, 1)] * 7
        params_net['initial_mu_inverse'] = 0.0

        params_net['initial_y1']= 0.8

        params_net['coef_mu_inverse'] = 0.36

        params_net['CalInGPU'] = gpu # whether to calculate in GPU
        params_net['kernel'] = params_net['kernel'][0:params_net['layers']]
        params_net['size1'] = 400
        params_net['size2'] = 500

    return params_net

In [ ]:
import scipy.io
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define custom PyTorch Dataset class
class CustomDataset(Dataset):
    def __init__(self, mat_file_M, mat_file_M_Omega):
        # Load the .mat files
        data_M = scipy.io.loadmat(mat_file_M)
        data_M_Omega = scipy.io.loadmat(mat_file_M_Omega)

        # Extract the data arrays
        self.M = torch.from_numpy(data_M[list(data_M.keys())[-1]]).float()
        self.M = self.M.permute(2, 0, 1)
        self.M_Omega = torch.from_numpy(data_M_Omega[list(data_M_Omega.keys())[-1]]).float()
        self.M_Omega = self.M_Omega.permute(2, 0, 1)

    def __len__(self):
        return len(self.M)

    def __getitem__(self, idx):
        return self.M[idx], self.M_Omega[idx]

# Define file paths for the training and test datasets
train_M_file = 'M_train1.mat'
train_M_Omega_file = 'M_Omega_train1.mat'
test_M_file = 'M_test1.mat'
test_M_Omega_file = 'M_Omega_test1.mat'

# Create CustomDataset instances for training and test data
train_dataset = CustomDataset(train_M_file, train_M_Omega_file)
test_dataset = CustomDataset(test_M_file, test_M_Omega_file)

# Create PyTorch DataLoader instances for training and test data
batch_size = 4  # You can adjust this as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example usage of the DataLoader
for batch in train_loader:
    M, M_Omega = batch
    # M contains ground truth data
    # M_Omega contains noisy/sampled data
    # You can perform your training or testing operations here


In [ ]:
M, M_Omega = next(iter(test_loader))
M.shape, M_Omega.shape

In [ ]:
# Some settings for visualisation
matplotlib.use('Agg')
%matplotlib inline

seed = 123
torch.manual_seed(seed)

# Set parameters (including hyperparameters) and setting for saving/logging data

hyper_param_net = training.get_hyperparameter_grid('ConvMC-Net', TrainInstances = 40, ValInstances = 20, BatchSize = 4, ValBatchSize = 4, Alpha = 1.0, num_epochs = 40, learning_rate = 0.012)

params_net = get_default_param(hyper_param_net, True)

CalInGPU = params_net['CalInGPU']

q_list = [0.5]
sigma_list = [3.0]

for q in q_list:
  for sigma in sigma_list:
    ProjectName = TRY + ' ' + logs_and_results.get_current_time() + ' ' + hyper_param_net['Model'] + ' ' + 'Sampling Rate: ' + logs_and_results.get_q_str(q) + ' and Noise Variance ' + logs_and_results.get_noise_str(sigma)

    # Get log file
    logfile = logs_and_results.get_modularized_record(ProjectName, q, sigma, 'Logs', hyper_param_net, params_net, SESSION)
    log = open(logfile, 'w')
    print('Project Name: %s'%ProjectName)
    log.write('Project Name: %s\n'%ProjectName)

    # Get Model
    net = training.get_model(params_net, hyper_param_net, log)
    print('Parameters = \n%s\n'%str(params_net))
    log.write('params_net=\n%s\n\n'%str(params_net))

    #Loading data and creating dataloader for both test and training
    print('Loading Data phase...')
    print('----------------')
    log.write('Loading phase...\n')
    log.write('----------------\n')
    shape_dset = (params_net['size1'], params_net['size2'])
    # train_dataset = dataset_processing.ImageDataset(round(hyper_param_net['TrainInstances']), shape_dset, 0, q, sigma)
    # train_loader = data.DataLoader(train_dataset,batch_size = hyper_param_net['BatchSize'], shuffle = True)
    # val_dataset = dataset_processing.ImageDataset(round(hyper_param_net['ValInstances']), shape_dset, 1, q, sigma)
    # test_loader = data.DataLoader(val_dataset, batch_size = hyper_param_net['ValBatchSize'], shuffle = True)
    print('Finished loading.\n')
    log.write('Finished loading.\n\n');

    # Some additional settings for training including loss, optimizer,
    floss = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr = hyper_param_net['Lr'])
    scheduler2 =  torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1, gamma = 0.97, verbose = True)

    # Array for recording parameter values after each layer for each epoch etc
    outputs_L = convmc.to_var(torch.zeros([shape_dset[0], shape_dset[1]]), CalInGPU)
    lossmean_vec = np.zeros((hyper_param_net['Epochs'], ))
    lossmean_val_vec = np.zeros((hyper_param_net['Epochs'], ))

    mu_inverse, y1, exp_L = net.getexp_LS()

    mu_inverse_vec = np.zeros((hyper_param_net['Epochs'], net.layers))
    y1_vec = np.zeros((hyper_param_net['Epochs'], net.layers,params_net['size1'], params_net['size2']))
    exp_L_vec = np.zeros((hyper_param_net['Epochs'], net.layers))

    # dummy variable to monitor and record progress for loss
    minloss = np.inf

    for epoch in range(hyper_param_net['Epochs']):
      print(f'Epoch: {epoch + 1}, {logs_and_results.get_current_time()}, \n')
      log.write('\n' + logs_and_results.get_current_time() + '\n')

      # Train and Test Steps. (Record every 5 epochs)
      if (epoch + 1) % 5 == 0:
          print('Loading and calculating training batches...')
          log.write('Loading and calculating training batches...\n')
          startime = time.time()
          loss_mean, loss_lowrank_mean = training.train_step(net, train_loader, floss, optimizer, CalInGPU, hyper_param_net['Alpha'], hyper_param_net['TrainInstances'], hyper_param_net['BatchSize'])
          endtime = time.time()
          print('Training time is %f'%(endtime - startime))
          log.write('Training time is %f\n'%(endtime - startime))

          print('Loading and calculating validation batches...')
          log.write('Loading and calculating validation batches...\n')
          startime = time.time()
          loss_val_mean, loss_val_lowrank_mean = training.test_step(net, test_loader, floss, optimizer, CalInGPU, hyper_param_net['Alpha'], hyper_param_net['ValInstances'], hyper_param_net['ValBatchSize'])
          endtime = time.time()
          print('Test time is %f'%(endtime - startime))
          log.write('Test time is %f\n'%(endtime - startime))
      else:
        loss_mean, loss_lowrank_mean = training.train_step(net, train_loader, floss, optimizer, CalInGPU, hyper_param_net['Alpha'], hyper_param_net['TrainInstances'], hyper_param_net['BatchSize'])
        loss_val_mean, loss_val_lowrank_mean = training.test_step(net, test_loader, floss, optimizer, CalInGPU, hyper_param_net['Alpha'], hyper_param_net['ValInstances'], hyper_param_net['ValBatchSize'])

      # Update Record and Parameters
      lossmean_vec[epoch] = loss_mean
      lossmean_val_vec[epoch] = loss_val_mean

      mu_inverse, y1, exp_L = net.getexp_LS()

      mu_inverse_vec[epoch, :] = mu_inverse
      y1_vec[epoch] = y1
      exp_L_vec[epoch, :] = exp_L

      # Update Log after every 5 epochs. Make a plot of MSE against epochs every 5 epochs. Save Model in whole/dict form every five epochs.
      if (epoch + 1) % 5 == 0:
        print(f"Saving Whole Model at Epochs: [{epoch + 1}/{hyper_param_net['Epochs']}]")
        model_whole_path = logs_and_results.get_modularized_record(ProjectName, q, sigma, 'Saved Models - Whole', hyper_param_net, params_net, SESSION, current_epoch = epoch + 1)
        torch.save(net, model_whole_path)
        print(f"Saving Model Dict at Epochs: [{epoch + 1}/{hyper_param_net['Epochs']}]")
        model_state_dict_path = logs_and_results.get_modularized_record(ProjectName, q, sigma, 'Saved Models - Dict', hyper_param_net, params_net, SESSION, current_epoch = epoch + 1)
        torch.save(net.state_dict(), model_state_dict_path)

        print('Epoch [%d/%d], Lossmean:%.5e, Validation lossmean:%.5e'
              %(epoch + 1, hyper_param_net['Epochs'], loss_mean, loss_val_mean))
        print('loss_lowrank_mean', loss_lowrank_mean)
        print('loss_val_lowrank_mean', loss_val_lowrank_mean)

        log.write('loss_lowrank_mean %.5e\n' %(loss_lowrank_mean))
        log.write('loss_val_lowrank_mean %.5e\n' %(loss_val_lowrank_mean))
        log.write('Epoch [%d/%d], Lossmean:%.5e, Validation lossmean:%.5e\n'
              %(epoch + 1, hyper_param_net['Epochs'], loss_mean, loss_val_mean))
        np.set_printoptions(precision = 3)

        print('mu_inverse:', mu_inverse)

        print('torch.mean(y1)', np.mean(y1))
        print('y1:', y1)
        print('exp_L:', exp_L)

        log.write('mu_inverse: '+ str(mu_inverse)+'\n')
        log.write('y1: '+ str(y1)+'\n')
        log.write('exp_L: '+ str(exp_L) + '\n')

        if True or loss_val_mean<minloss:
          print('saved at [epoch%d/%d]'%(epoch + 1, hyper_param_net['Epochs']))
          log.write('saved at [epoch%d/%d]\n' %(epoch + 1, hyper_param_net['Epochs']))
          minloss = min(loss_val_mean, minloss)

        # Plotting MSE vs Epoch and Saving it

        # Get Directory where we have to save the plot
        dir = logs_and_results.get_modularized_record(ProjectName, q, sigma, 'Plots', hyper_param_net, params_net, SESSION, current_epoch = epoch + 1)
        epochs_vec = np.arange(0, hyper_param_net['Epochs'], 1)
        logs_and_results.plot_and_save_mse_vs_epoch(epochs_vec, lossmean_vec, hyper_param_net, lossmean_val_vec, dir, epoch)

    # Finish off by observing the minimum loss on validation set

    #Print min loss
    print('\nmin Loss = %.4e'%np.min(lossmean_val_vec))
    log.write('\nmin Loss = %.4e\n'%np.min(lossmean_val_vec))
    log.close()

Project Name: 5st try 2023-10-04 00:11:23 ConvMC-Net Sampling Rate: 50.0% and Noise Variance 3.0
Configuring Network...

Instantiating Model...

Model Instantiated...

Parameters = 
{'layers': 5, 'kernel': [(3, 1), (3, 1), (3, 1), (3, 1), (3, 1)], 'initial_mu_inverse': 0.0, 'initial_y1': 0.8, 'coef_mu_inverse': 0.36, 'CalInGPU': True, 'size1': 400, 'size2': 500}

Loading Data phase...
----------------
Finished loading.

Adjusting learning rate of group 0 to 1.2000e-02.
Epoch: 1, 2023-10-04 00:11:23, 



RuntimeError: ignored

In [ ]:
ROOT

'/content/drive/MyDrive/DUPA - RCPA/Technology transfer deep unfolding/SPROJ-ConvMC-Net/Sensor Data'

In [ ]:
!cp -r '/content/drive/MyDrive/DUPA - RCPA/Technology transfer deep unfolding/SPROJ-ConvMC-Net/Sensor Data' '/content/convmc-net'
